In [14]:
# -*- codeing = utf-8 -*-
from bs4 import BeautifulSoup           # 网页解析
import requests     # URL操作，获取网页数据
import pandas as pd
from sqlalchemy import create_engine
import re

# 请求伪装成浏览器
hearders = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}

In [19]:
names = ['光之子','狂神','空速星痕','琴帝']
res = []
for name in names:
    # url = f'https://www.douban.com/search?cat=1002&q={name}' # 仅影视
    url = f'https://www.douban.com/search?q={name}'
    # 获取查询结果，解析为soup
    request = requests.get(url,headers=hearders)
    soup = BeautifulSoup(request.text, "html.parser")
    # 获取查询结果列表
    information_list = soup.find('div', class_='result-list').find_all('div',class_='result')
    li = []
    for information in information_list:
        # 获取详细信息链接
        if name not in information.find('h3').find('a').text:
            continue
        information_url = information.find('h3').find('a')['href']
        # 获取详细信息页面，解析为soup
        request_inf = requests.get(information_url,headers=hearders)
        soup_inf = BeautifulSoup(request_inf.text, "html.parser")
        # 定位信息标签
        info = soup_inf.find('div',id='info')
        # 获取信息标签列表
        try:
            pl = info.find_all('span',class_='pl')
        except Exception as e:
            print(information.find('h3').find('a').text,'获取信息标签列表错误',e)
            continue
        # 解析信息，生成字典
        dic = {}
        dic['系列'] = name
        dic['名称'] = soup_inf.find('span',property='v:itemreviewed').text
        for i in pl:
            key =re.sub(r'[ :]','',i.text)
            if i.parent.name == 'span':
                try:
                    dic[key] = i.parent.find('span',class_='attrs').text
                except Exception as e:
                    dic[key] = i.parent.find('a').text
            elif i.next_sibling.next_sibling.name == "a":
                dic[key] = i.next_sibling.next_sibling.text.replace('\n','')
                print(i.next_sibling.next_sibling)
            elif i.text == '类型:':
                genre = ''
                for j in info.find_all('span',property='v:genre'):
                    genre = genre+j.text+'/'
                dic[key] = genre[:-1]
            elif i.text == '首播:' or i.text == '上映日期:':
                dic[key] = info.find('span',property='v:initialReleaseDate').text[:10]
            elif i.text == '片长:':
                dic[key] = info.find('span',property='v:runtime').text
            else:
                dic[key] = i.next_sibling.replace(' ','')
        li.append(dic)
    res.extend(li)

df = pd.DataFrame(res)
df

光之子 Child of Light 获取信息标签列表错误 'NoneType' object has no attribute 'find_all'
<a href="https://book.douban.com/press/2146">湖南文艺出版社</a>
<a href="https://book.douban.com/author/4565283/">
            唐家三少</a>
<a href="https://book.douban.com/press/2124">百花洲文艺出版社</a>
<a href="https://book.douban.com/series/14499">狂神</a>
<a href="https://book.douban.com/author/4565283/">
            唐家三少</a>
<a href="https://book.douban.com/press/2319">北方文艺出版社</a>
<a href="https://book.douban.com/author/4565283/">
            唐家三少</a>
<a href="https://book.douban.com/press/2808">太白文艺出版社</a>
<a href="https://book.douban.com/author/4565283/">
            唐家三少</a>
<a href="https://book.douban.com/press/2808">太白文艺出版社</a>
<a href="https://book.douban.com/series/14500">琴帝</a>


,系列,名称,作者,出版社,出版年,页数,定价,装帧,ISBN,导演,编剧,类型,制片国家/地区,语言,上映日期,片长,又名,IMDb,副标题,丛书
0,光之子,光之子,唐家三少,湖南文艺出版社,2018-7-1,1096,CNY118.00,平装,9787540483067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,光之子,光之子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,卡先加,卡先加,纪录片,中国大陆,汉语普通话,2020-07-29,100分钟,DaughteroftheLight,tt12630666,NaN,NaN
2,狂神,狂神,唐家三少,百花洲文艺出版社,2005-12,273,22.00元,NaN,9787806479223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,龙神帝国,狂神
3,空速星痕,空速星痕,唐家三少,北方文艺出版社,2006-09-22,NaN,22.00元,NaN,9787531720249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,琴帝,琴帝,唐家三少,太白文艺出版社,2008-05,267,26.00元,平装,9787806806081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,六道之决,NaN
5,琴帝,琴帝1·千手琴音,唐家三少,太白文艺出版社,2012-4,299,28.00元,平装,9787551301602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,琴帝


In [20]:
df.columns

Index(['系列', '名称', '作者', '出版社', '出版年', '页数', '定价', '装帧', 'ISBN', '导演', '编剧',
       '类型', '制片国家/地区', '语言', '上映日期', '片长', '又名', 'IMDb', '副标题', '丛书'],
      dtype='object')

In [29]:
n = pd.read_table('name.txt')
for i in n['name']:
    print(i)

光之子
狂神
空速星痕
琴帝
冰火魔厨
惟我独仙
酒神
天珠变
为了你我愿意热爱整个世界
天火大道
神印王座
斗罗大陆
不死不灭
神墓
遮天
完美世界
魔兽剑圣异界纵横
斗破苍穹
武动乾坤
大主宰
元尊
万相之王
寸芒
九鼎记
星辰变
盘龙
吞噬星空
莽荒纪
雪鹰领主
飞剑问道
沧元图
诛仙
